<a href="https://colab.research.google.com/github/aetev/Hearth-Stone-Python-Simulator/blob/main/card_enc_types.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json
import tensorflow as tf
import re
from tensorflow.keras.layers import Embedding, TextVectorization
# Assuming AtomicCards.json is in your Google Drive, adjust the path if necessary
file_path = '/content/drive/MyDrive/MTGdata/AtomicCards.json'


with open(file_path, 'r') as f:
  data = json.load(f)['data']

In [3]:
def filter_legal_cards(data):
    legal_commander_data = {}
    for card_name, card_data in data.items():
        try:
            if card_data[0]['legalities']['commander'] == 'Legal':
                legal_commander_data[card_name] = card_data[0] # Store only the first element of card_data
        except (KeyError, IndexError):
            pass  # Or print a message, log it, etc.
    return legal_commander_data

filtered_data = filter_legal_cards(data)


In [4]:
items = list(filtered_data.items())


type_list = []
for row in items[:100]:
  second_key, second_value = row
  if second_value['type'] != None:
    print(second_value['type'])
    type_list.append(second_value['type'])

Creature — Zombie Guest
Creature — Goblin Guest
Artifact — Equipment
Instant
Enchantment
Instant
Enchantment
Creature — Bird Rogue
Legendary Creature — Insect Druid
Legendary Creature — Astartes Warrior
Sorcery
Instant
Sorcery
Land
Land
Artifact
Creature — Zombie
Creature — Gargoyle
Creature — Griffin
Creature — Human Cleric
Creature — Human Monk
Legendary Creature — Human Warrior
Enchantment — Aura
Creature — Tyranid Mutant
Creature — Human Elf Shaman
Creature — Human Insect
Instant
Creature — Eye
Creature — Demon
Enchantment
Instant
Instant
Creature — Fish Horror
Instant
Snow Creature — Treefolk
Creature — Horror
Creature — Horror
Legendary Creature — Elf Horror
Creature — Elemental
Enchantment — Aura
Legendary Creature — Octopus Noble
Instant
Instant
Land — Desert
Instant
Enchantment
Enchantment
Creature — Thrull Cleric
Creature — Lammasu
Instant
Instant
Sorcery
Legendary Land
Instant
Creature — Avatar
Instant
Creature — Human
Sorcery
Legendary Creature — Spirit
Creature — Cat Cleri

In [5]:
print(type_list[:150])

['Creature — Zombie Guest', 'Creature — Goblin Guest', 'Artifact — Equipment', 'Instant', 'Enchantment', 'Instant', 'Enchantment', 'Creature — Bird Rogue', 'Legendary Creature — Insect Druid', 'Legendary Creature — Astartes Warrior', 'Sorcery', 'Instant', 'Sorcery', 'Land', 'Land', 'Artifact', 'Creature — Zombie', 'Creature — Gargoyle', 'Creature — Griffin', 'Creature — Human Cleric', 'Creature — Human Monk', 'Legendary Creature — Human Warrior', 'Enchantment — Aura', 'Creature — Tyranid Mutant', 'Creature — Human Elf Shaman', 'Creature — Human Insect', 'Instant', 'Creature — Eye', 'Creature — Demon', 'Enchantment', 'Instant', 'Instant', 'Creature — Fish Horror', 'Instant', 'Snow Creature — Treefolk', 'Creature — Horror', 'Creature — Horror', 'Legendary Creature — Elf Horror', 'Creature — Elemental', 'Enchantment — Aura', 'Legendary Creature — Octopus Noble', 'Instant', 'Instant', 'Land — Desert', 'Instant', 'Enchantment', 'Enchantment', 'Creature — Thrull Cleric', 'Creature — Lammasu'

In [ ]:
from typing_extensions import Self
class TypeEncoderLayer(tf.keras.layers.Layer):
    def __init__(self,card_dict, **kwargs):
        super(TypeEncoderLayer, self).__init__(**kwargs)
        self.vectorizer = TextVectorization(standardize=None, output_mode='multi_hot')

        type_list = [types for card_name, card_data in card_dict.items() if card_data['type'] is not None for types in card_data['type']]
        cleaned_type_list = self.clean_text(tf.constant(type_list))
        self.vectorizer.adapt(cleaned_type_list)
        self.embedding_layer = Embedding(input_dim=3000, output_dim=100)

    def clean_text(self, text):
        cleaned_text = tf.strings.regex_replace(text, '[^a-zA-Z0-9\s]', '')
        cleaned_text = tf.strings.lower(cleaned_text)
        return cleaned_text

    def call(self, inputs):
        cleaned_text = self.clean_text(inputs)
        vectorized_text = self.vectorizer(cleaned_text)
        embedded_text = self.embedding_layer(vectorized_text)
        return cleaned_text


# Example usage with a batch of texts
batch_texts = tf.constant(type_list)
cleaned_text_layer = TypeEncoderLayer(filtered_data)
cleaned_batch = cleaned_text_layer.call(batch_texts)
for i in range(len(cleaned_batch)):
  tf.print(cleaned_batch[i])